# **Run all of these codes using a GPU**

# Test for Math

In [ ]:
  from transformers import pipeline
  from langchain_huggingface import HuggingFacePipeline
  from langchain.prompts import PromptTemplate
  from transformers.utils.logging import set_verbosity_error

  set_verbosity_error()

  # Use Phi-2 for math solving
  math_pipeline = pipeline(
      "text-generation",
      model="microsoft/phi-2", # hjskhan/gemma-2b-fine-tuned-math
      device=0,
      max_new_tokens=256,  # 💡 increase for full explanation
      temperature=0.7,
      do_sample=True
  )

  math_solver = HuggingFacePipeline(pipeline=math_pipeline)

  # QA model (same as before)
  qa_pipeline = pipeline("question-answering", model="bert-large-uncased-whole-word-masking-finetuned-squad", device=-1)

  # Prompt to force step-by-step reasoning
  math_template = PromptTemplate.from_template(
      "You are a math expert. Solve the following problem step-by-step and explain clearly:\n\n{problem}\n\nSolution:"
  )

  # Chain definition
  math_chain = math_template | math_solver

  # Ask for math problem
  problem = input("\nEnter a math problem (e.g. 'What is the derivative of x^2 + 3x?'):\n")

  # Generate the answer
  solution = math_chain.invoke({"problem": problem})

  print("\n🔹 **Solution Explanation:**")
  print(solution)

  # QA loop
  while True:
      question = input("\nAsk a question about the solution (or type 'exit' to stop):\n")
      if question.lower() == "exit":
          break

      qa_result = qa_pipeline(question=question, context=solution)++++
      print("\n🔹 **Answer:**")
      print(qa_result["answer"])


# Test for Summarization

In [ ]:
from transformers import pipeline
from langchain_huggingface import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from transformers.utils.logging import set_verbosity_error

set_verbosity_error()

summarization_pipeline = pipeline("summarization", model="facebook/bart-large-cnn", device=0)
summarizer = HuggingFacePipeline(pipeline=summarization_pipeline)

refinement_pipeline = pipeline("summarization", model="facebook/bart-large", device=0)
refiner = HuggingFacePipeline(pipeline=refinement_pipeline)

qa_pipeline = pipeline("question-answering", model="deepset/roberta-base-squad2", device=0)

summary_template = PromptTemplate.from_template("Summarize the following text in a {length} way:\n\n{text}")

summarization_chain = summary_template | summarizer | refiner

text_to_summarize = input("\nEnter text to summarize:\n")
length = input("\nEnter the length (short/medium/long): ")

summary = summarization_chain.invoke({"text": text_to_summarize, "length": length})

print("\n🔹 **Generated Summary:**")
print(summary)

while True:
    question = input("\nAsk a question about the summary (or type 'exit' to stop):\n")
    if question.lower() == "exit":
        break

    qa_result = qa_pipeline(question=question, context=summary)

    print("\n🔹 **Answer:**")
    print(qa_result["answer"])